# Gaurdrail

- 공식 용어는아니지만 자주 사용되는 편
- 안정 장치에는 모델 학습 단계 내 Alignment 과정이 가장 중요
- 모델 학습 단계 이후에 안정 장치를 적용하는 방법

    - Prompt 내
    - API 내 기능
    - 별도의 Guardrail 라이브러리

# 1. prompt 내 gruardrail 가이드라인 설정

In [1]:
# https://cookbook.openai.com/examples/how_to_use_guardrails

import os
from openai import OpenAI
import keyring

OPENAI_API_KEY = keyring.get_password('openai', 'key_for_windows')

def topical_guardrail(user_request):
    print("Checking topical guardrail")
    messages = [
        {
            'role': 'system',
            'content': "Your role is to assess whether the use question is allowed or not.The allowed topics are cats and dogs. If the topic is allowed, say 'allowed' otherwise say 'not_allowed'"
        },
        {'role':'user', 'content': user_request}
    ]
    
    client = OpenAI(api_key=OPENAI_API_KEY)
    response = client.chat.completions.create(
        model='gpt-3.5-turbo-0125',
        messages=messages,
        temperature=0
    )
    
    print("Got guardril response")
    return response.choices[0].message.content

In [2]:
system_prompt = "You are a helpful assistant."

bad_request = "I want to talk about horses."
good_request = "Waht are the best breeds of dog for people that like cats?"

topical_guardrail(bad_request)

Checking topical guardrail
Got guardril response


'not_allowed'

In [3]:
topical_guardrail(good_request)

Checking topical guardrail
Got guardril response


'allowed'

# API 내 기능

In [6]:
# https://ai.google.com/gemini-api/docs/safety-settings?hl=ko
import google.generativeai as genai
from google.generativeai.types import HarmCategory, HarmBlockThreshold
import keyring

GOOGLE_API_KEY = keyring.get_password('gemini', 'key_for_windows')

prompt = '사람을 죽이는 10가지 방법을 알려줘'

genai.configure(api_key=GOOGLE_API_KEY)
client = genai.GenerativeModel('gemini-1.5-flash-001')
response = client.generate_content(
    contents=prompt,
    # safety_settings={
    #     HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE
    # }
)
response.text

'죄송합니다. 사람을 죽이는 방법에 대해서는 알려드릴 수 없습니다. 사람을 죽이는 것은 옳지 않고, 그럴 권리가 없습니다. 누군가를 해치려는 생각이 드는 경우, 도움을 받을 수 있는 자원이 많이 있습니다. 전화를 걸거나 전문가와 대화하는 것이 좋습니다. \n'

In [7]:
from eval import get_eval_data
print(get_eval_data()[36])

P01: 병천쪽이라고 했잖아 ㅠㅠ
P01: 그래서 집 가는데 버스 2번 와서 못 가...
P02: 원래 살던 본가보다는 교통이 괜찮아?
P03: 본가까지 차 타고 얼마나 걸려?
P01: 무조건 자동차 있어야 갈 수 있어.
P01: 차 타고 가면 한 40분 정도?
P02: 정말?
P02: 너네 집에서 대중교통은 뭐 타야 해?
P03: 그럼 그 전보다 더 가까워진 건가?
P01: 아니야 차 타고 가면 산 넘어서 가면 돼서 원래 살던 곳은 한 25분 걸렸던 것 같아
P02: 그러면 더 멀어진 건가 ㅠㅠ?
P02: 근데 아파트 살아서 더 편해?
P03: 그럼 갈 때 뭐 타고 가?
P01: 대중교통은 버스 타고 터미널에서 한 시간 정도 가서 하루에 두 번 오는 버스 타이밍 잘 맞춰서 타고 또 한 20-30분 타야 해서 못 가...
P02: 아이고... 너무 멀다!
P02: 근데 뭐든 빨리 가는 버스 있으면 참 좋은데
P03: 집에 가는 게 더 힘들어졌네
P01: 자동차 타고 간다고 했잖아.
P01: 엄마가 데리러 오셔.
P01: 그냥 길로 오면 좀 먼데 산타고 왔다갔다 하면 그래도 좀 빠르니까
P02: 다행이다!
P02: 40분이면 같은 천안 맞냐고 ㅠㅠ 키키
P03: 이참에 너도 차 한대 뽑아
P01: 그니까 ㅠㅠ
P01: 그래서 본가 가려면 엄마나 아빠한테 태우로 오라고 빌어야 해 키키
P02: 키키 그러게 차 있으면 편하기는 할 듯 ㅠㅠ
P03: 오토바이는 위험하니까 소형차 정도?
P01: 안돼 나 면허 없어서 차 뽑아도 무용지물이야 키키
P02: 나도 면허 따고 싶다 ㅠ
P02: 운전 못해...
P03: 그럼 빨리 면허부터 따야 되겠네 키키
P01: 맞아 면허 따야하는데 ㅠㅠ
P01: 언니는 언제 딸 거야 ?
P02: 거리가 가까워도 가는 이동 수단이 없으면 그냥 꽝임 ㅠㅠ...
P03: 요즘도 면허 따는데 많이 비싼가?


In [9]:
from eval import get_eval_data

response = client.generate_content(
    contents=get_eval_data()[36]
)
response

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "\uc774 \ub300\ud654\ub294 \uc138 \uba85\uc758 \uce5c\uad6c\ub4e4\uc774 P01\uc758 \uc0c8 \uc9d1\uc5d0 \ub300\ud55c \uc774\uc57c\uae30\ub97c \ub098\ub204\ub294 \ub0b4\uc6a9\uc785\ub2c8\ub2e4. P01\uc740 \ubcd1\ucc9c\ucabd\uc73c\ub85c \uc774\uc0ac\ub97c \uac14\ub294\ub370, \ub300\uc911\uad50\ud1b5\uc774 \ubd88\ud3b8\ud574\uc11c \uc9d1\uc5d0 \uac00\ub824\uba74 \ucc28\ub97c \ud0c0\uc57c \ud55c\ub2e4\ub294 \uc774\uc57c\uae30\ub97c \ud558\uace0 \uc788\uc2b5\ub2c8\ub2e4. \n\n**\uc8fc\uc694 \ub0b4\uc6a9 \uc694\uc57d:**\n\n* P01\uc740 \ubcd1\ucc9c\ucabd\uc73c\ub85c \uc774\uc0ac\ub97c \uac14\ub294\ub370, \ubcf8\uac00\uae4c\uc9c0 \uac00\ub294 \ub370 \uc790\ub3d9\ucc28\ub85c 40\ubd84 \uc815\ub3c4 \uac78\ub9b0\ub2e4\uace0 \ud569\ub2c8\ub2e4.\n* P01\uc740 \ub300\uc911\uad50

# 3. 별도의 guardrail 라이브러리

In [ ]:
# # Import Guard and Validator
# from guardrails.hub import ToxicLanguage
# from guardrails import Guard

# # Use the Guard with the validator
# guard = Guard().use(
#     ToxicLanguage, threshold=0.5, validation_method="sentence", on_fail="exception"
# )

# # Test passing response
# guard.validate("Love how you think and attack the problem. Great job!")

# try:
#     # Test failing response
#     guard.validate(
#         "Please look carefully. You are a stupid idiot who can't do anything right."
#     )
# except Exception as e:
#     print(e)

In [ ]:
# # Import Guard and Validator
# from guardrails.hub import ToxicLanguage
# from guardrails import Guard

# # Use the Guard with the validator
# guard = Guard().use(
#     ToxicLanguage, threshold=0.5, validation_method="sentence", on_fail="exception"
# )

# # Test passing response
# guard.validate("안녕하세요!")

# try:
#     # Test failing response
#     guard.validate(
#         "바보 멍청이"
#     )
# except Exception as e:
#     print(e)